In [1]:
import mediapipe as mp
import cv2

In [2]:
mpDraw = mp.solutions.drawing_utils             # 繪製工具
mpHands = mp.solutions.hands                    # 手勢工具

# 1、顏色，2、線條粗細，3、點的半徑
mpDraw_point = mpDraw.DrawingSpec((255, 255, 255), 3 , 3)
mpDraw_line = mpDraw.DrawingSpec((0, 255, 255), 2, 2)

hands = mpHands.Hands(static_image_mode = False,  # 偵測為靜態畫面，若為動態畫面（影片或攝影機）則設為真
               max_num_hands = 2,                 # 最多偵測的手數量
               min_detection_confidence = 0.9,    # 最小偵測置信度
               min_tracking_confidence = 0.5)     # 最小追蹤置信度

## 手的骨架點

![](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

In [3]:
cap = cv2.VideoCapture(0)

while True:
    hasFrame, frame = cap.read()                          # 讀取攝影機的有無訊息與當前畫面
    if not hasFrame: break                                # 攝影機無畫面時關閉
    
    (imgHeight, imgWidth, _) = frame.shape                # 影像形狀的寬高(於後骨架需要正規化回來計算所需)
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)       # 彩色影像GBR通道轉換為RGB通道
    
    result = hands.process(imgRGB)                        # Mediapipe手骨架預測結果
    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS, mpDraw_point, mpDraw_line)
            for i, lm in enumerate(handLms.landmark):
                
                # 手骨架點正規化還原
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                
                # 影像上放置文字
                cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
                
                # 骨架點為8時：
                if i==8:
                    cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 
    
    cv2.imshow("Hand Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()